# 「同志」怎麼了？

上次我們發現了當「同志」一詞作為複合名詞的前半部時，在兩個語料中有不同的分佈情況，尤其是下福盟的語料當中有些有趣的用法。這次我想看的是當句子出現同志一詞時，所使用的動詞有什麼不同。常說動詞/述詞就像串起句子元素的主線，句子中的許多成分好比該動詞所表達的動作的參與角色。同志既可以作為受事者被霸凌、被忽視、被壓迫，也可以作為主事者站出來、進行抗爭、要求權利，因此我好奇兩個語料中的「同志」是否參與了不同動作，成為不同故事/敘事中的主角。

以下我將練習用 `jieba.posseg` 所提供的詞性標注功能，針對兩個語料當中，包含「同志」的句子進行詞性標註，並篩選出所有的動詞。

In [18]:
import nltk
import re
import json
import jieba
import jieba.posseg as pseg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

jieba.set_dictionary('jieba_dict/dict.txt.big')
jieba.load_userdict('jieba_dict/user_dict.txt')

plt.rcParams['figure.figsize'] = [10, 5]
myfont = FontProperties(fname=r'/usr/local/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/msj.ttf',size=14)
sns.set(font=myfont.get_name())

%matplotlib inline

pd.set_option('display.max_rows', 2000)

LGBTFAMILY_PATH = "raw_content_pro.json"
LOVEFAMILY_PATH = "raw_content_anti.json"

lgbtfamily_raw_text = ""
lovefamily_raw_text = ""

with open(LGBTFAMILY_PATH) as f:
    lgbtfamily_raw_text = json.loads(f.read())

with open(LOVEFAMILY_PATH) as f:
    lovefamily_raw_text = json.loads(f.read())

Building prefix dict from /Users/koukantou/Desktop/Don/ntu_master/107-2-lab-meeting/nano-blog/jieba_dict/dict.txt.big ...
Dumping model to file cache /var/folders/vp/g4gpl3x91qqdn6_x0t8w4bbc0000gn/T/jieba.u336b009ae6d5715d70945d8db9630e20.cache
Loading model cost 2.718 seconds.
Prefix dict has been built succesfully.


In [5]:
# 前處理函式
def generate_line_sentence(all_text):  
    # 以句號分句
    all_text = re.split('[。，！？!?,]', all_text)
    
    # 去除所有非中文字元
    all_text = [_get_chinese(s) for s in all_text]
    
    return all_text

In [34]:
# 接收語料(dict)與關鍵詞(string)，回傳語料當中關鍵詞出現的句子中，按照頻率排列的動詞表
def generate_verb_list(raw_text_list, word):
    a = [generate_line_sentence(article['content']) for article in raw_text_list]
    l = [sen for article in a for sen in article if word in sen]
    l = [list(pseg.cut(ll)) for ll in l]
    wordlist = [word for sen in l for (word, pos) in sen if pos.startswith('v')]
    freq_dict = {w:wordlist.count(w) for w in set(wordlist)}
    return sorted(freq_dict.items(), key=lambda kv: -kv[1])

# 接收語料(dict)與關鍵詞(string)，回傳語料中含有該關鍵詞的concordance dataframe
def generate_concordance_df(raw_text_list, word):
    a = [generate_line_sentence(article['content']) for article in raw_text_list]
    l = [sen.split(word) for article in a for sen in article if word in sen]
    df = pd.DataFrame({
    "left_context": [s[0] for s in l],
    "keyword": word,
    "right_context": [s[1] for s in l],
    })
    return df

## 伴侶盟中的同志

先看一下伴侶盟語料中含有「同志」的句子：

In [35]:
generate_concordance_df(lgbtfamily_raw_text, '同志')

,left_context,keyword,right_context
0,反觀他的,同志,朋友在台灣某銀行當客服人員
1,協助被剝奪結婚自由的台灣,同志,
2,並期待大法官能藉此矯正我國民法對於,同志,的制度性歧視
3,祁家威為台灣第一位公開出櫃的,同志,
4,並於年就開始要求政府保障,同志,結婚的權益
5,年過去,同志,權益逐漸擴張
6,痛批法律竟如此公開踐踏,同志,人權
7,盼為憲法守門員的大法官能夠還給,同志,平等的人權
8,還給,同志,平等的婚姻權益捍衛
9,原文連結蕭婷方自由時報為保障,同志,權益與跨性別權益


再看伴侶盟語料中，經過`jieba`詞性標注後，所有含有「同志」一次句子中的動詞列表：

In [31]:
pd.DataFrame(generate_verb_list(lgbtfamily_raw_text, '同志'))

,0,1
0,教育,79
1,是,74
2,有,45
3,公投,41
4,保障,32
5,禁止,26
6,要,21
7,會,17
8,支持,15
9,要求,13


## 下福盟中的同志

先看下福盟語料中含有「同志」的句子：

In [36]:
generate_concordance_df(lovefamily_raw_text, '同志')

,left_context,keyword,right_context
0,同性婚姻理由,同志,族群因社會歧視汙名而不易現身
1,但各國研究均顯示,同志,家庭佔人口一定比例
2,根據在網路上發起的真愛聖誕,同志,路過教會公民行動中提到我想到了一個活動計劃
3,就是號召全臺灣的,同志,與直
4,或提供給司法檢調單位作為呈堂證供的依據真愛聖誕,同志,路過教會公民行動時間年月日日地點臺灣各地教會會址時間依各教會公告時間人數以人為集結單位
5,,同志,朋友去了幾間教會後
6,很多教會也都表明歡迎,同志,朋友
7,我們相信同運團體所有的,同志,朋友
8,難怪教宗會說同運團體不好但很多,同志,朋友被他們給污名化了提供教宗影片難道台灣沒有信仰自由嗎
9,干擾聚會我們相信同運團體所有的,同志,朋友


再看下福盟語料中，經過`jieba`
詞性標注後，所有含有「同志」一次句子中的動詞列表：

In [32]:
pd.DataFrame(generate_verb_list(lovefamily_raw_text, '同志'))

,0,1
0,教育,255
1,是,157
2,有,89
3,要,45
4,公投,45
5,保障,27
6,支持,25
7,會,24
8,研究,22
9,到,20


## 比較
乍看之下，`jieba.posseg`所標註的結果並沒有想像中完美，有很多像是複合名詞的後半部被標示為動詞，例如「同志教育」中的「教育」被標示為動詞，這顯示了中文詞性標注的困難，也展現了結巴詞性標注的一些限制。在這個過程中，我也想過會不會將句子丟入dependency parser，更能夠看出句子當中「同志」與其他句子成分的關聯性，當然這需要更多實驗。

之前常常看的是高頻出現的詞語，但我發現那些只出現一兩次的詞語也滿有趣的，大家可以再回頭看看。

以下舉兩個動詞為例。

### 同志「感染」
「感染」一詞在伴侶盟語料沒有出現，在下福盟則出現了5次。此詞的出現，強調了同志作為疾病的載體。或許不難在媒體上看到類似「同性戀感染某種疾病」的言論，但反觀想想「異性戀感染某種疾病」這句話，是否便沒有那麼自然。疾病不分性向，但當疾病作為令人恐懼的符碼，並與特定團體連結上時，或許我們應該去思考這是否就是一種污名，是對疾病的污名，也是對特定群體的污名。

### 「禁止」同志
「禁止」一詞在伴侶盟語料出現了26次，在下福盟則出現了6次。「同志」作為被禁止的群體在伴侶盟的語料中大量出現，顯示了伴侶盟的語料中，常常將「同志」作為受事者的角色，是被特定組織所「禁止」和被特定宗教所「反對」的行為與群體。

另外，在下福盟的語料關於同志的語句當中，出現了不少「嗑」、「淪」、「吸毒」、「染黑」的動詞，也常常能看到「向孩子們『散佈』同志養成教育」的句子，可以看到他們試圖讓同志族群與某一類動詞不斷共現，將同志貼上負面的標籤，並使用情緒煽動性較高的字眼。

## 小結
無論同志作為主動「散佈」同志教育的角色，或者同志作為被「禁止」的角色，我們可以看到針對同一個詞語，不同說話者運用不同的動詞使之參與不同的語意角色，能夠從中做一些有趣的觀察。如果先撇開運用程式找出同志作為施事者或受事者的方法，我用肉眼掃過一遍上述語料後，能大抵看到伴侶盟中的「同志」常常被塑造成受事者、而下福盟中的「同志」常常被形容成行動主事者的現象。這就好比（隨便舉個政治上的例子）台灣在國際上的情勢，臺灣方的自我論述常常是受到打壓、漠視、被叫成中國台灣的，但如果是中共看台灣的角度，則常常能看到類似「台灣破壞什麼原則」、「違背什麼共識」的論述。但我上面的觀察仍需要以更可操作化的方式提出證明。

這篇以結巴做的初步嘗試，邊做的時候就邊一直想到底有什麼更好的方式，可以找出與「同志」相關連的動詞。大概想到三種其他方法：

1. Dependency parser: 我不確定透過dependency parser是否能更精準的抓出「同志」一詞在句子當中與其他成分的依賴關係，但也許可以跑看看。
2. WordSketch: 後來發現我想做的事似乎能運用WordSketch找出，只是我還在摸索如何對自己收集的語料下WordSketch的query。請參考：[中研院所提供的中文ＷordSketch](http://wordsketch.ling.sinica.edu.tw/)
3. WordNet: 既然我想看的是與「同志」一詞相關連的動詞使用情形，或許能運用現有的中文詞網資源，去檢視兩個語料中的動詞在詞網中是否呈現什麼樣的模式、在意義網路中是否能看什麼樣的星系分佈。不過目前我對詞網還不太熟悉，或許這是一個日後可以好好瞭解的方向。